In [1]:
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

SentenceBERT 모델 로드

In [2]:
model = SentenceTransformer('jhgan/ko-sroberta-multitask')

sentences = ["안녕하세요?", "한국어 문장 임베딩을 위한 버트 모델입니다."]
embeddings = model.encode(sentences)

print(embeddings)

[[-0.37510446 -0.77338433  0.5927712  ...  0.57923514  0.32683456
  -0.6508967 ]
 [-0.09361697 -0.18191518 -0.19230804 ... -0.03165806  0.30412525
  -0.2679361 ]]


In [3]:
df = pd.read_csv('computer.csv')

df.head()

,품명,가격,링크
0,삼성전자 삼성 DM500TDA-A38A,"741,040원",https://adcr.naver.com/adcr?x=lRY2h8N2wcCFgin2...
1,삼성전자 삼성 DM500SDZ-AD5A,"729,000원",https://adcr.naver.com/adcr?x=yR/6t66fjsTCfIaK...
2,한성컴퓨터 TFG MX9465S,"878,000원",https://adcr.naver.com/adcr?x=XG0vA8W5qu0g8MZO...
3,삼성전자 삼성 DM530ADA-L15A,"592,630원",https://cr.shopping.naver.com/adcr.nhn?x=kYYcN...
4,삼성전자 삼성 DM530ADA-L78A,"1,333,660원",https://cr.shopping.naver.com/adcr.nhn?x=MCgi3...


전처리

In [4]:
df = df.drop(columns=['링크'])

df.head()

,품명,가격
0,삼성전자 삼성 DM500TDA-A38A,"741,040원"
1,삼성전자 삼성 DM500SDZ-AD5A,"729,000원"
2,한성컴퓨터 TFG MX9465S,"878,000원"
3,삼성전자 삼성 DM530ADA-L15A,"592,630원"
4,삼성전자 삼성 DM530ADA-L78A,"1,333,660원"


In [5]:
df.loc[0, '품명']

'삼성전자 삼성 DM500TDA-A38A'

In [6]:
model.encode(df.loc[0, '품명'])

array([ 4.54589397e-01,  2.62408823e-01, -6.83214590e-02, -3.06486394e-02,
       -1.82356179e-01, -4.14297998e-01,  4.32798356e-01, -1.78386584e-01,
       -3.08389157e-01,  2.77258903e-01, -1.69698462e-01,  2.68526673e-01,
       -4.24423963e-01,  3.56342345e-01, -8.08300078e-02,  3.05231333e-01,
       -1.68940946e-01,  5.08443236e-01, -1.05127610e-01, -1.36436343e+00,
       -1.28231108e-01,  5.15903309e-02,  4.82359588e-01,  1.05233319e-01,
        1.95239082e-01,  3.16972554e-01, -1.64200366e-01, -2.88950354e-01,
        2.06479326e-01, -2.11310666e-02,  1.43728718e-01, -3.24559897e-01,
        1.41150326e-01, -6.94961488e-01,  1.52255744e-01, -4.46220130e-01,
       -4.03074771e-01, -2.87983455e-02, -5.64190924e-01, -8.04996431e-01,
        2.07443744e-01, -3.39649385e-03,  2.83288598e-01,  3.35778266e-01,
        8.02419126e-01, -2.34468862e-01,  2.73429662e-01,  2.20359161e-01,
        3.80189687e-01, -2.30993032e-01, -2.10753366e-01,  1.01641506e-01,
       -2.75167134e-02, -

유저 대화내용 인코딩

In [7]:
df['embedding'] = pd.Series([[]] * len(df)) # dummy

df['embedding'] = df['품명'].map(lambda x: list(model.encode(x)))

df.head()

,품명,가격,embedding
0,삼성전자 삼성 DM500TDA-A38A,"741,040원","[0.4545894, 0.26240882, -0.06832146, -0.030648..."
1,삼성전자 삼성 DM500SDZ-AD5A,"729,000원","[0.45490846, 0.34968647, 0.22412069, -0.355774..."
2,한성컴퓨터 TFG MX9465S,"878,000원","[-0.4821302, 0.0239543, 0.024152769, -0.442421..."
3,삼성전자 삼성 DM530ADA-L15A,"592,630원","[0.43843973, 0.36715037, 0.5040723, -0.5082708..."
4,삼성전자 삼성 DM530ADA-L78A,"1,333,660원","[0.37922537, 0.44932634, 0.28506044, -0.277239..."


In [8]:
df.to_csv('computer1.csv', index=False)

간단한 챗봇

In [9]:
text = '삼성전자 삼성 DM530ADA-L78A'

embedding = model.encode(text)

In [10]:
df['similarity'] = df['embedding'].map(lambda x: cosine_similarity([embedding], [x]).squeeze())

df.head()

,품명,가격,embedding,similarity
0,삼성전자 삼성 DM500TDA-A38A,"741,040원","[0.4545894, 0.26240882, -0.06832146, -0.030648...",0.850676
1,삼성전자 삼성 DM500SDZ-AD5A,"729,000원","[0.45490846, 0.34968647, 0.22412069, -0.355774...",0.865888
2,한성컴퓨터 TFG MX9465S,"878,000원","[-0.4821302, 0.0239543, 0.024152769, -0.442421...",0.462458
3,삼성전자 삼성 DM530ADA-L15A,"592,630원","[0.43843973, 0.36715037, 0.5040723, -0.5082708...",0.876439
4,삼성전자 삼성 DM530ADA-L78A,"1,333,660원","[0.37922537, 0.44932634, 0.28506044, -0.277239...",1.000000


In [11]:
answer = df.loc[df['similarity'].idxmax()] # 최대값 뽑기

print('챗봇 답변', answer['가격'])
print('유사도', answer['similarity'])

챗봇 답변 1,333,660원
유사도 1.0000001192092896


In [12]:
text = 'IGNIQ 비와이 30Y'

embedding = model.encode(text)

In [13]:
df['similarity'] = df['embedding'].map(lambda x: cosine_similarity([embedding], [x]).squeeze())

df.head()

,품명,가격,embedding,similarity
0,삼성전자 삼성 DM500TDA-A38A,"741,040원","[0.4545894, 0.26240882, -0.06832146, -0.030648...",0.278538
1,삼성전자 삼성 DM500SDZ-AD5A,"729,000원","[0.45490846, 0.34968647, 0.22412069, -0.355774...",0.172590
2,한성컴퓨터 TFG MX9465S,"878,000원","[-0.4821302, 0.0239543, 0.024152769, -0.442421...",0.311813
3,삼성전자 삼성 DM530ADA-L15A,"592,630원","[0.43843973, 0.36715037, 0.5040723, -0.5082708...",0.211780
4,삼성전자 삼성 DM530ADA-L78A,"1,333,660원","[0.37922537, 0.44932634, 0.28506044, -0.277239...",0.181291


In [14]:
answer = df.loc[df['similarity'].idxmax()]

print('챗봇 답변', answer['가격'])
print('유사도', answer['similarity'])

챗봇 답변 1,099,000원
유사도 1.0


In [15]:
df

,품명,가격,embedding,similarity
0,삼성전자 삼성 DM500TDA-A38A,"741,040원","[0.4545894, 0.26240882, -0.06832146, -0.030648...",0.278538
1,삼성전자 삼성 DM500SDZ-AD5A,"729,000원","[0.45490846, 0.34968647, 0.22412069, -0.355774...",0.172590
2,한성컴퓨터 TFG MX9465S,"878,000원","[-0.4821302, 0.0239543, 0.024152769, -0.442421...",0.311813
3,삼성전자 삼성 DM530ADA-L15A,"592,630원","[0.43843973, 0.36715037, 0.5040723, -0.5082708...",0.211780
4,삼성전자 삼성 DM530ADA-L78A,"1,333,660원","[0.37922537, 0.44932634, 0.28506044, -0.277239...",0.181291
...,...,...,...,...
822,디클 A24-310 윈도우,"699,000원","[-0.21041104, 0.34711185, 0.4194438, 0.2322478...",0.318181
823,삼성데스크탑본체 i5 8GB SSD512GB 정품윈도우11 삼성컴퓨터본체 사무용컴퓨터,"1,018,890원","[-0.16718023, -0.06351796, 0.10895853, 0.24156...",0.185421
824,인텔 11세대 i5-11400/8G/256G/컴집조립PC/가정.사무용,"397,440원","[0.15457909, -0.040598445, -0.25815114, -0.459...",0.116212
825,Apple 아이맥 24형 2021년형 M1 8코어 CPU 및 8코어 GPU 256G...,"1,806,140원","[-0.36709404, 0.06672264, 0.014325685, 0.36360...",0.249845


In [16]:
text = '삼성전자 삼성 DM500SDZ-AD5A'

embedding = model.encode(text)

In [17]:
df['similarity'] = df['embedding'].map(lambda x: cosine_similarity([embedding], [x]).squeeze())

df.head()

,품명,가격,embedding,similarity
0,삼성전자 삼성 DM500TDA-A38A,"741,040원","[0.4545894, 0.26240882, -0.06832146, -0.030648...",0.834929
1,삼성전자 삼성 DM500SDZ-AD5A,"729,000원","[0.45490846, 0.34968647, 0.22412069, -0.355774...",1.000000
2,한성컴퓨터 TFG MX9465S,"878,000원","[-0.4821302, 0.0239543, 0.024152769, -0.442421...",0.497394
3,삼성전자 삼성 DM530ADA-L15A,"592,630원","[0.43843973, 0.36715037, 0.5040723, -0.5082708...",0.895725
4,삼성전자 삼성 DM530ADA-L78A,"1,333,660원","[0.37922537, 0.44932634, 0.28506044, -0.277239...",0.865888


In [18]:
answer = df.loc[df['similarity'].idxmax()]

print('챗봇 답변', answer['가격'])
print('유사도', answer['similarity'])

챗봇 답변 729,000원
유사도 0.9999998807907104


In [19]:
text = '사무용 컴퓨터 데스크탑 10세대'

embedding = model.encode(text)

In [20]:
df['similarity'] = df['embedding'].map(lambda x: cosine_similarity([embedding], [x]).squeeze())

df.head()

,품명,가격,embedding,similarity
0,삼성전자 삼성 DM500TDA-A38A,"741,040원","[0.4545894, 0.26240882, -0.06832146, -0.030648...",0.326444
1,삼성전자 삼성 DM500SDZ-AD5A,"729,000원","[0.45490846, 0.34968647, 0.22412069, -0.355774...",0.309521
2,한성컴퓨터 TFG MX9465S,"878,000원","[-0.4821302, 0.0239543, 0.024152769, -0.442421...",0.478732
3,삼성전자 삼성 DM530ADA-L15A,"592,630원","[0.43843973, 0.36715037, 0.5040723, -0.5082708...",0.297984
4,삼성전자 삼성 DM530ADA-L78A,"1,333,660원","[0.37922537, 0.44932634, 0.28506044, -0.277239...",0.305203


In [21]:
answer = df.loc[df['similarity'].idxmax()]

print('챗봇 답변', answer['가격'])
print('유사도', answer['similarity'])

챗봇 답변 739,000원
유사도 0.7755223512649536


In [22]:
df

,품명,가격,embedding,similarity
0,삼성전자 삼성 DM500TDA-A38A,"741,040원","[0.4545894, 0.26240882, -0.06832146, -0.030648...",0.326444
1,삼성전자 삼성 DM500SDZ-AD5A,"729,000원","[0.45490846, 0.34968647, 0.22412069, -0.355774...",0.309521
2,한성컴퓨터 TFG MX9465S,"878,000원","[-0.4821302, 0.0239543, 0.024152769, -0.442421...",0.478732
3,삼성전자 삼성 DM530ADA-L15A,"592,630원","[0.43843973, 0.36715037, 0.5040723, -0.5082708...",0.297984
4,삼성전자 삼성 DM530ADA-L78A,"1,333,660원","[0.37922537, 0.44932634, 0.28506044, -0.277239...",0.305203
...,...,...,...,...
822,디클 A24-310 윈도우,"699,000원","[-0.21041104, 0.34711185, 0.4194438, 0.2322478...",0.435975
823,삼성데스크탑본체 i5 8GB SSD512GB 정품윈도우11 삼성컴퓨터본체 사무용컴퓨터,"1,018,890원","[-0.16718023, -0.06351796, 0.10895853, 0.24156...",0.633082
824,인텔 11세대 i5-11400/8G/256G/컴집조립PC/가정.사무용,"397,440원","[0.15457909, -0.040598445, -0.25815114, -0.459...",0.681575
825,Apple 아이맥 24형 2021년형 M1 8코어 CPU 및 8코어 GPU 256G...,"1,806,140원","[-0.36709404, 0.06672264, 0.014325685, 0.36360...",0.465614


In [23]:
text = ' 사무용컴퓨터'

embedding = model.encode(text)

In [24]:
df['similarity'] = df['embedding'].map(lambda x: cosine_similarity([embedding], [x]).squeeze())

df.head()

,품명,가격,embedding,similarity
0,삼성전자 삼성 DM500TDA-A38A,"741,040원","[0.4545894, 0.26240882, -0.06832146, -0.030648...",0.320978
1,삼성전자 삼성 DM500SDZ-AD5A,"729,000원","[0.45490846, 0.34968647, 0.22412069, -0.355774...",0.305800
2,한성컴퓨터 TFG MX9465S,"878,000원","[-0.4821302, 0.0239543, 0.024152769, -0.442421...",0.563971
3,삼성전자 삼성 DM530ADA-L15A,"592,630원","[0.43843973, 0.36715037, 0.5040723, -0.5082708...",0.312021
4,삼성전자 삼성 DM530ADA-L78A,"1,333,660원","[0.37922537, 0.44932634, 0.28506044, -0.277239...",0.305867


In [25]:
answer = df.loc[df['similarity'].idxmax()]

print('챗봇 답변', answer['가격'])
print('유사도', answer['similarity'])

챗봇 답변 805,000원
유사도 0.7380939722061157


In [26]:
df

,품명,가격,embedding,similarity
0,삼성전자 삼성 DM500TDA-A38A,"741,040원","[0.4545894, 0.26240882, -0.06832146, -0.030648...",0.320978
1,삼성전자 삼성 DM500SDZ-AD5A,"729,000원","[0.45490846, 0.34968647, 0.22412069, -0.355774...",0.305800
2,한성컴퓨터 TFG MX9465S,"878,000원","[-0.4821302, 0.0239543, 0.024152769, -0.442421...",0.563971
3,삼성전자 삼성 DM530ADA-L15A,"592,630원","[0.43843973, 0.36715037, 0.5040723, -0.5082708...",0.312021
4,삼성전자 삼성 DM530ADA-L78A,"1,333,660원","[0.37922537, 0.44932634, 0.28506044, -0.277239...",0.305867
...,...,...,...,...
822,디클 A24-310 윈도우,"699,000원","[-0.21041104, 0.34711185, 0.4194438, 0.2322478...",0.432733
823,삼성데스크탑본체 i5 8GB SSD512GB 정품윈도우11 삼성컴퓨터본체 사무용컴퓨터,"1,018,890원","[-0.16718023, -0.06351796, 0.10895853, 0.24156...",0.650922
824,인텔 11세대 i5-11400/8G/256G/컴집조립PC/가정.사무용,"397,440원","[0.15457909, -0.040598445, -0.25815114, -0.459...",0.614997
825,Apple 아이맥 24형 2021년형 M1 8코어 CPU 및 8코어 GPU 256G...,"1,806,140원","[-0.36709404, 0.06672264, 0.014325685, 0.36360...",0.358972


In [27]:
df

,품명,가격,embedding,similarity
0,삼성전자 삼성 DM500TDA-A38A,"741,040원","[0.4545894, 0.26240882, -0.06832146, -0.030648...",0.320978
1,삼성전자 삼성 DM500SDZ-AD5A,"729,000원","[0.45490846, 0.34968647, 0.22412069, -0.355774...",0.305800
2,한성컴퓨터 TFG MX9465S,"878,000원","[-0.4821302, 0.0239543, 0.024152769, -0.442421...",0.563971
3,삼성전자 삼성 DM530ADA-L15A,"592,630원","[0.43843973, 0.36715037, 0.5040723, -0.5082708...",0.312021
4,삼성전자 삼성 DM530ADA-L78A,"1,333,660원","[0.37922537, 0.44932634, 0.28506044, -0.277239...",0.305867
...,...,...,...,...
822,디클 A24-310 윈도우,"699,000원","[-0.21041104, 0.34711185, 0.4194438, 0.2322478...",0.432733
823,삼성데스크탑본체 i5 8GB SSD512GB 정품윈도우11 삼성컴퓨터본체 사무용컴퓨터,"1,018,890원","[-0.16718023, -0.06351796, 0.10895853, 0.24156...",0.650922
824,인텔 11세대 i5-11400/8G/256G/컴집조립PC/가정.사무용,"397,440원","[0.15457909, -0.040598445, -0.25815114, -0.459...",0.614997
825,Apple 아이맥 24형 2021년형 M1 8코어 CPU 및 8코어 GPU 256G...,"1,806,140원","[-0.36709404, 0.06672264, 0.014325685, 0.36360...",0.358972


In [28]:
text = ' A24-310 '

embedding = model.encode(text)

In [29]:
df['similarity'] = df['embedding'].map(lambda x: cosine_similarity([embedding], [x]).squeeze())

df.head()

,품명,가격,embedding,similarity
0,삼성전자 삼성 DM500TDA-A38A,"741,040원","[0.4545894, 0.26240882, -0.06832146, -0.030648...",0.472705
1,삼성전자 삼성 DM500SDZ-AD5A,"729,000원","[0.45490846, 0.34968647, 0.22412069, -0.355774...",0.413657
2,한성컴퓨터 TFG MX9465S,"878,000원","[-0.4821302, 0.0239543, 0.024152769, -0.442421...",0.427432
3,삼성전자 삼성 DM530ADA-L15A,"592,630원","[0.43843973, 0.36715037, 0.5040723, -0.5082708...",0.382098
4,삼성전자 삼성 DM530ADA-L78A,"1,333,660원","[0.37922537, 0.44932634, 0.28506044, -0.277239...",0.396787


In [30]:
answer = df.loc[df['similarity'].idxmax()]

print('챗봇 답변', answer['가격'])
print('유사도', answer['similarity'])

챗봇 답변 699,000원
유사도 0.6007322072982788


In [31]:
df

,품명,가격,embedding,similarity
0,삼성전자 삼성 DM500TDA-A38A,"741,040원","[0.4545894, 0.26240882, -0.06832146, -0.030648...",0.472705
1,삼성전자 삼성 DM500SDZ-AD5A,"729,000원","[0.45490846, 0.34968647, 0.22412069, -0.355774...",0.413657
2,한성컴퓨터 TFG MX9465S,"878,000원","[-0.4821302, 0.0239543, 0.024152769, -0.442421...",0.427432
3,삼성전자 삼성 DM530ADA-L15A,"592,630원","[0.43843973, 0.36715037, 0.5040723, -0.5082708...",0.382098
4,삼성전자 삼성 DM530ADA-L78A,"1,333,660원","[0.37922537, 0.44932634, 0.28506044, -0.277239...",0.396787
...,...,...,...,...
822,디클 A24-310 윈도우,"699,000원","[-0.21041104, 0.34711185, 0.4194438, 0.2322478...",0.600732
823,삼성데스크탑본체 i5 8GB SSD512GB 정품윈도우11 삼성컴퓨터본체 사무용컴퓨터,"1,018,890원","[-0.16718023, -0.06351796, 0.10895853, 0.24156...",0.387941
824,인텔 11세대 i5-11400/8G/256G/컴집조립PC/가정.사무용,"397,440원","[0.15457909, -0.040598445, -0.25815114, -0.459...",0.227867
825,Apple 아이맥 24형 2021년형 M1 8코어 CPU 및 8코어 GPU 256G...,"1,806,140원","[-0.36709404, 0.06672264, 0.014325685, 0.36360...",0.367418
